In [2]:
from pprint import pprint

# dataset: https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews
# Product Id	User Id	Score	Summary	Text
import transformers
import plotly
import matplotlib
import sklearn
import torch

import torchvision
import scipy

In [6]:
import pandas as pd
# ProductId	UserId	Score	Summary	Text

df = pd.read_csv("../data/Reviews.csv")
# display Reviews.csv
df.head(20)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
5,6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,4,1342051200,Nice Taffy,I got a wild hair for taffy and ordered this f...
6,7,B006K2ZZ7K,A1SP2KVKFXXRU1,David C. Sullivan,0,0,5,1340150400,Great! Just as good as the expensive brands!,This saltwater taffy had great flavors and was...
7,8,B006K2ZZ7K,A3JRGQVEQN31IQ,Pamela G. Williams,0,0,5,1336003200,"Wonderful, tasty taffy",This taffy is so good. It is very soft and ch...
8,9,B000E7L2R4,A1MZYO9TZK0BBI,R. James,1,1,5,1322006400,Yay Barley,Right now I'm mostly just sprouting this so my...
9,10,B00171APVA,A21BT40VZCCYT4,Carol A. Reed,0,0,5,1351209600,Healthy Dog Food,This is a very healthy dog food. Good for thei...


In [ ]:
import pandas as pd
import tiktoken

from openai import OpenAI
client = OpenAI()

def get_embedding(text, model="text-embedding-3-small"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input = [text], model=model).data[0].embedding


df['ada_embedding'] = df.combined.apply(lambda x: get_embedding(x, model='text-embedding-3-small'))
df.to_csv('output/embedded_1k_reviews.csv', index=False)

embedding_model = "text-embedding-3-small"
embedding_encoding = "cl100k_base"
max_tokens = 8000  # the maximum for text-embedding-3-small is 8191

# load & inspect dataset
input_datapath = "data/fine_food_reviews_1k.csv"  # to save space, we provide a pre-filtered dataset
df = pd.read_csv(input_datapath, index_col=0)
df = df[["Time", "ProductId", "UserId", "Score", "Summary", "Text"]]
df = df.dropna()
df["combined"] = (
    "Title: " + df.Summary.str.strip() + "; Content: " + df.Text.str.strip()
)
df.head(2)

In [6]:
import pandas as pd
import json
"""
※Pandas の read_json は、デフォルトでは「配列 of オブジェクト」や「テーブル形式」を期待する　ので、
(1) 標準の json モジュールで読み込んで
(2) data.items() からキー／値ペアを DataFrame に変換し
(3) あわせてキー一覧を取得
"""
json_file = 'agents_docs.json'

with open(json_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

df = pd.DataFrame(
    list(data.items()),
    columns=['key', 'value']
)

print(df.head(10))
print('-----------')

print(df['key'].to_list())

doc_key_list = df['key'].to_list()

import pprint

pprint.pprint(doc_key_list)

print(data['Agents'])


                            key  \
0                         Intro   
1                    Quickstart   
2                      Examples   
3                        Agents   
4                Running agents   
5                       Results   
6                     Streaming   
7                         Tools   
8  Model context protocol (MCP)   
9                      Handoffs   

                                               value  
0  OpenAI Agents SDK\nThe\nOpenAI Agents SDK\nena...  
1  Quickstart\nPrerequisites\nMake sure you've fo...  
2  Examples\nCheck out a variety of sample implem...  
3  Agents\nToolsToFinalOutputFunction\nmodule-att...  
4  Running agents\nYou can run agents via the\nRu...  
5  Results\nRunResultBase\ndataclass\nBases:\nABC...  
6  Streaming\nStreaming lets you subscribe to upd...  
7  Tools\nTool\nmodule-attribute\nTool\n=\nUnion\...  
8  Model context protocol (MCP)\nThe\nModel conte...  
9  Handoffs\nHandoffInputFilter\nmodule-attribute...  
---------

KeyError: 'Agents'